In [1]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
# Merge datasets for lookalike model
data = pd.merge(transactions, customers, on='CustomerID')
data = pd.merge(data, products, on='ProductID')

# Verify column names
print("Columns in data after merge:", data.columns)

# Ensure the Price column exists or calculate it
if 'Price' not in data.columns:
    data['Price'] = data['TotalValue'] / data['Quantity']

# Preprocessing for Lookalike Model
customer_features = data.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'Quantity': 'sum',
    'Price': 'mean',
}).reset_index()

customer_features = pd.merge(customer_features, customers[['CustomerID', 'Region']], on='CustomerID')
customer_features['Region'] = LabelEncoder().fit_transform(customer_features['Region'])

# Normalize data
scaler = StandardScaler()
feature_matrix = scaler.fit_transform(customer_features.drop(columns=['CustomerID']))

# Build Lookalike Model (KNN)
knn = NearestNeighbors(n_neighbors=4, metric='cosine')  # 3 similar customers + self
knn.fit(feature_matrix)

# Find top 3 similar customers for first 20 customers
lookalike_results = {}
for i in range(20):
    distances, indices = knn.kneighbors([feature_matrix[i]])
    lookalike_results[customer_features.iloc[i]['CustomerID']] = [
        (customer_features.iloc[indices[0][j]]['CustomerID'], 1 - distances[0][j])
        for j in range(1, 4)
    ]